### [主要參考文章 WOE](https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html#Steps-of-Calculating-WOE)
是一種很直觀的計算方法，容易解讀及調整。故在銀行界當中被大量使用，其中 WOE 的計算公式是其核心計算方法。
* Cacluation fo WOE
<img src="https://3.bp.blogspot.com/-eqZJpJZ4Kig/VPnZUBaP7II/AAAAAAAADkc/yeW8XVL35dA/s1600/weight%2Bof%2Bevidence.png" />
<img src="https://i.imgur.com/vw8eTbI.png"  width=50% />

* Example Caculation of WOE
<img src="https://i.imgur.com/koSCN4u.png" />
* Rule of WOE
<img src="https://i.imgur.com/vw8eTbI.png" width=50% />

###  重點白話說明
* 計算一個在因子(細切成每一個 bin) postive 的佔比，與 negative 的佔比
* 在切 Bin 時 positve 、 negative 佔比必需有一定的差距
* 最後 將 X 根據Bin 的分佈所對照出來的 WOE 所查找出來，並將 Sum 起，就是預測值 Y
* Peseudo Code
<img src="https://i.imgur.com/59w8udc.png" width="25%" />


In [1]:
#Load Required Packages
import pandas as pd
import numpy as np

In [18]:
#Read Data
# we want to predict this "admit"
data = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

In [19]:
data.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [20]:
data.describe()

,admit,gre,gpa,rank
count,400.000000,400.000000,400.000000,400.00000
mean,0.317500,587.700000,3.389900,2.48500
std,0.466087,115.516536,0.380567,0.94446
min,0.000000,220.000000,2.260000,1.00000
25%,0.000000,520.000000,3.130000,2.00000
50%,0.000000,580.000000,3.395000,2.00000
75%,1.000000,660.000000,3.670000,3.00000
max,1.000000,800.000000,4.000000,4.00000


In [10]:
#Empty Dataframe
newDF,woeDF = pd.DataFrame(), pd.DataFrame()

In [5]:
cols = data.columns
print(cols)
target = 'admit'
bins = 10

Index(['admit', 'gre', 'gpa', 'rank'], dtype='object')


### pd.qcut 函式說明
* 根據 bin number 來將原始的輸入資料切分到所對應的 bin lable
* e.q. (620.0, 660.0] ps: closed 為右 => 620 < value <=660

In [22]:
for ivars in cols[~cols.isin([target])]:
#     print(ivars)
    if(data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
#         print(data[ivars].dtype.kind)
#         print(ivars)
        binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
        
#         print(binned_x)
        
        d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        
#         print(d0)
        
        break

i
gre


In [26]:
def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF

### 計算 IV 值
IV = ∑ (% of non-events - % of events) * WOE


In [27]:
#Read Data
mydata = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")
iv, woe = iv_woe(data = mydata, target = 'admit', bins=10, show_woe = True)


Information value of gre is 0.312882
  Variable            Cutoff   N  Events  % of Events  Non-Events  \
0      gre  (219.999, 440.0]  48       6     0.047244          42   
1      gre    (440.0, 500.0]  51      12     0.094488          39   
2      gre    (500.0, 520.0]  24      10     0.078740          14   
3      gre    (520.0, 560.0]  51      15     0.118110          36   
4      gre    (560.0, 580.0]  29       6     0.047244          23   
5      gre    (580.0, 620.0]  53      21     0.165354          32   
6      gre    (620.0, 660.0]  45      17     0.133858          28   
7      gre    (660.0, 680.0]  20       9     0.070866          11   
8      gre    (680.0, 740.0]  44      12     0.094488          32   
9      gre    (740.0, 800.0]  35      19     0.149606          16   

   % of Non-Events       WoE        IV  
0         0.153846 -1.180625  0.125857  
1         0.142857 -0.413370  0.019994  
2         0.051282  0.428812  0.011774  
3         0.131868 -0.110184  0.001516 

In [28]:
iv

,Variable,IV
0,gre,0.312882
0,gpa,0.270020
0,rank,0.292044


In [29]:
woe

,Variable,Cutoff,N,Events,% of Events,Non-Events,% of Non-Events,WoE,IV
0,gre,"(219.999, 440.0]",48,6,0.047244,42,0.153846,-1.180625,0.125857
1,gre,"(440.0, 500.0]",51,12,0.094488,39,0.142857,-0.413370,0.019994
2,gre,"(500.0, 520.0]",24,10,0.078740,14,0.051282,0.428812,0.011774
3,gre,"(520.0, 560.0]",51,15,0.118110,36,0.131868,-0.110184,0.001516
4,gre,"(560.0, 580.0]",29,6,0.047244,23,0.084249,-0.578450,0.021406
5,gre,"(580.0, 620.0]",53,21,0.165354,32,0.117216,0.344071,0.016563
6,gre,"(620.0, 660.0]",45,17,0.133858,28,0.102564,0.266294,0.008333
7,gre,"(660.0, 680.0]",20,9,0.070866,11,0.040293,0.564614,0.017262
8,gre,"(680.0, 740.0]",44,12,0.094488,32,0.117216,-0.215545,0.004899
9,gre,"(740.0, 800.0]",35,19,0.149606,16,0.058608,0.937135,0.085278


### Do Predction at Exit Data
gre:380   => woe=-1.180625
gpa:3.61  => woe=0.318998
rank:3    => woe=0.052295

sum of woe = -1.180625 + 0.318998 + 0.052295 = -0.809332

so result of admiit is fail 


In [33]:
mydata.head(10)

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
5,1,760,3.00,2
6,1,560,2.98,1
7,0,400,3.08,2
8,1,540,3.39,3
9,0,700,3.92,2
